<a href="https://colab.research.google.com/github/urness/CS167Fall2025/blob/main/Weighted_kNNSolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS167: Day07
##Weighted kNN Solution

#### CS167: Machine Learning, Fall 2025


## Before we get started, let's load in our datasets:
Make sure you change the path to match your Google Drive.


In [1]:
import pandas as pd
import numpy as np

# The first step is to mount your Google Drive to your Colab account.
#You will be asked to authorize Colab to access your Google Drive. Follow the steps they lead you.

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#import the data:
#make sure the path on the line below corresponds to the path where you put your dataset.
iris_df = pd.read_csv('/content/drive/MyDrive/CS167/datasets/irisData.csv')

#Normalize each of the predictor columns in the iris dataset.

In [3]:
#petal_length
pl_mean = iris_df['petal length'].mean()
pl_std = iris_df['petal length'].std()

#petal_width
pw_mean = iris_df['petal width'].mean()
pw_std = iris_df['petal width'].std()

#sepal_length
sl_mean = iris_df['sepal length'].mean()
sl_std = iris_df['sepal length'].std()

#sepal_width
sw_mean = iris_df['sepal width'].mean()
sw_std = iris_df['sepal width'].std()

#make a copy of the original dataframe
normalized_iris_df = iris_df.copy()

#replace column with each entry's z-score
normalized_iris_df['petal length'] = (normalized_iris_df['petal length'] - pl_mean)/pl_std
normalized_iris_df['petal width'] = (normalized_iris_df['petal width'] - pw_mean)/pw_std
normalized_iris_df['sepal length'] = (normalized_iris_df['sepal length'] - sl_mean)/sl_std
normalized_iris_df['sepal width'] = (normalized_iris_df['sepal width'] - sw_mean)/sw_std
normalized_iris_df.head()

,sepal length,sepal width,petal length,petal width,species
0,-0.897674,1.028611,-1.336794,-1.308593,Iris-setosa
1,-1.139200,-0.124540,-1.336794,-1.308593,Iris-setosa
2,-1.380727,0.336720,-1.393470,-1.308593,Iris-setosa
3,-1.501490,0.106090,-1.280118,-1.308593,Iris-setosa
4,-1.018437,1.259242,-1.336794,-1.308593,Iris-setosa


In [4]:
def kNN(specimen, data, k):
    # 1. calculate distances
    data['distance_to_new'] = np.sqrt(
     (specimen['petal length'] - data['petal length'])**2
    +(specimen['sepal length'] - data['sepal length'])**2
    +(specimen['petal width'] - data['petal width'])**2
    +(specimen['sepal width'] - data['sepal width'])**2)

    # 2. sort
    sorted_data = data.sort_values(['distance_to_new'])

    # 3. predict
    prediction = sorted_data.iloc[0:k]['species'].mode()[0]

    return prediction

In [5]:
def weighted_kNN(specimen, data, k):

    # step 1. calculate the distances from 'specimen' to all other samples in 'data'
    data['distance_to_new'] = np.sqrt(
    (specimen['petal length'] - data['petal length'])**2
    +(specimen['sepal length'] - data['sepal length'])**2
    +(specimen['petal width'] - data['petal width'])**2
    +(specimen['sepal width'] - data['sepal width'])**2)

    # step 2 calcuate the weights (remember, weights are 1/d^2)
    data['weights'] = 1/(data['distance_to_new']**2)

    # step 3. find the k closest neighbors as follows
    # first, sort the data and take the first k samples as neighbors
    sorted_data = data.sort_values(['distance_to_new'])
    neighbors = sorted_data.iloc[0:k]

    # second, use groupby to sum the weights of each species in the closest k
    results = neighbors.groupby('species')['weights'].sum()
    sorted_results = results.sort_values(ascending=False)
    # step 4. return the class that has the largest sum of weight.
    return sorted_results.index[0]

In [6]:
#classify this iris
new_iris = {}
new_iris['petal length'] = 5.1
new_iris['sepal length'] = 7.2
new_iris['petal width'] = 1.5
new_iris['sepal width'] = 2.5

# original data
print("ORIGINAL DATA:", kNN(new_iris, iris_df, 10))

#normalize the new iris
normalized_new_iris = {}
normalized_new_iris['petal length'] = (5.1 - pl_mean)/pl_std
normalized_new_iris['sepal length'] = (7.2 - sl_mean)/sl_std
normalized_new_iris['petal width'] = (1.5 - pw_mean)/pw_std
normalized_new_iris['sepal width'] = (2.5 - sw_mean)/sw_std

# normalized data
print("NORMALIZED DATA:", kNN(normalized_new_iris, normalized_iris_df, 10))

ORIGINAL DATA: Iris-versicolor
NORMALIZED DATA: Iris-virginica


In [16]:
# unweighted knn not normalized; k=3
print(kNN(new_iris, iris_df,3))

Iris-versicolor


In [17]:
# unweighted knn normalized; k=3
print(kNN(normalized_new_iris, normalized_iris_df,3))

Iris-virginica


In [18]:
# weighted knn not normalized; k=3
print(weighted_kNN(new_iris, iris_df,3))

Iris-versicolor


In [21]:
# weighted knn normalized; k=3
print(weighted_kNN(normalized_new_iris, normalized_iris_df,3))

Iris-virginica
